In [1]:
#python system libraries
import time
import sys
sys.path.append(r'C:\Users\Kingdel\Desktop\python\mcc_examples')

#daq libraries
from mcculw import ul
from mcculw.enums import DigitalIODirection
from examples.console import util
from examples.props.digital import DigitalProps
from mcculw.ul import ULError
from examples.props.ai import AnalogInputProps

#serial libraries
import time
import serial
ser=serial.Serial('COM3',timeout=3.0)
ser.flushInput()

#libraries for interaction, plotting and data analysis
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from ipywidgets import widgets
from bokeh.plotting import figure,show
from bokeh.io import output_notebook, push_notebook
output_notebook()

Loading BokehJS ...

In [2]:
#detect devices
board_num=0

ul.ignore_instacal()
if not util.config_first_detected_device(board_num):
    print("Could not find device.")


Found device: USB-202 (1D55213)



In [3]:
#set port information
digital_props = DigitalProps(board_num)

port = next(
    (port for port in digital_props.port_info
     if port.supports_output), None)
if port == None:
    util.print_unsupported_example(board_num)

# If the port is configurable, configure it for output.
if port.is_port_configurable:
    ul.d_config_port(board_num, port.type, DigitalIODirection.OUT)

port_value = 0xFF

print("Setting " + port.type.name + " to " + str(port_value) + ".")

# Output the value to the port
ul.d_out(board_num, port.type, port_value)

bit_num = 0 #CH0 --> normally high
bit_value = 1 ##set 5V to HIGH, power the sensor
print("Setting " + port.type.name + " bit " + str(bit_num) + " to "+ str(bit_value) + ".")

# Output the value to the bit
ul.d_bit_out(board_num, port.type, bit_num, bit_value)

# time.sleep(1) #some weird timing thing with mccdaq ul

Setting AUXPORT to 255.
Setting AUXPORT bit 0 to 1.


In [4]:
##set analog acquisition params
channel = 0
ai_props = AnalogInputProps(board_num)
ai_range = ai_props.available_ranges[0]

In [5]:
# Get a single value from the device - turn this into a test and throw exception if err

# Use the a_in method for devices with a resolution <= 16
value = ul.a_in(board_num, channel, ai_range)
# Convert the raw value to engineering units
eng_units_value = ul.to_eng_units(board_num, ai_range, value)


# Display the raw value
print("Raw Value: " + str(value))
# Display the engineering value
print("Engineering Value: " + '{:.3f}'.format(eng_units_value))



Raw Value: 2536
Engineering Value: 2.383


In [6]:
#dmm setup

def dmm_cmd(string,serial,delay):

    cmd=string+'\r\n'
    byte=cmd.encode('utf-8')
    time.sleep(delay)
    serial.write(byte)
    ser_bytes = serial.readline()
    data=ser_bytes.decode('utf-8')
    return data

In [7]:
print ('enter filename for test run')
text=widgets.Text()
display(text)

def handle_submit(sender):
    print('filename is {0}.csv'.format(text.value))

    
text.on_submit(handle_submit)

button=widgets.Button(description='start data logging')
display(button)

logging=True

def when_clicked(b):
    p1=figure(plot_width=650,plot_height=350)
    t=show(p1,notebook_handle=True)
    loop_num=300

    for i in range(loop_num):
#     while logging:
        
    
        # Use the a_in method for devices with a resolution <= 16
        value = ul.a_in(board_num, channel, ai_range)
        eng_units_value = ul.to_eng_units(board_num, ai_range, value) # Convert the raw value to engineering units
        
        #potentiometer resistance
        cmd='READ?'
        res=dmm_cmd(cmd,ser,0.1)
        res=float(res.strip()) #get rid of whitespace

        y=eng_units_value
        r1=p1.circle(res,y,size=10)
        push_notebook(handle=t) #https://bokeh.pydata.org/en/latest/docs/user_guide/notebook.html

        time.sleep(0.1)
    
button.on_click(when_clicked)

button2=widgets.Button(description='stop data logging')
display(button2)

def when_clicked(c):
    button2.close()
    print('step 3')
    print('step 4')
    
    var=5
    return var
    
test=button2.on_click(when_clicked)

enter filename for test run


Text(value='')

Button(description='start data logging', style=ButtonStyle())

Button(description='stop data logging', style=ButtonStyle())

In [8]:
# ul.release_daq_device(board_num)
# ser.close()